In [75]:
import numpy as np
import pandas as pd
import math
import pandas_datareader as web
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')


In [76]:
df = pd.read_csv('BTC-USD.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,16-07-2010,0.04951,0.04951,0.04951,0.04951,0.04951,0
1,17-07-2010,0.04951,0.08585,0.05941,0.08584,0.08584,5
2,18-07-2010,0.08584,0.09307,0.07723,0.08080,0.08080,49
3,19-07-2010,0.08080,0.08181,0.07426,0.07474,0.07474,20
4,20-07-2010,0.07474,0.07921,0.06634,0.07921,0.07921,42


In [47]:
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
3196,16-04-2019,5212.810059,5270.759766,5190.810059,5236.259766,5236.259766,194699629
3197,17-04-2019,5236.259766,5317.779785,5232.990234,5289.750000,5289.750000,187312150
3198,18-04-2019,5289.750000,5353.009766,5212.180176,5295.529785,5295.529785,158699049
3199,19-04-2019,5295.529785,5366.600098,5274.450195,5326.180176,5326.180176,123577001
3200,20-04-2019,5326.180176,5348.589844,5232.770020,5305.740234,5305.740234,134681237


In [24]:
from keras.engine import training
data = df.filter(['Close'])
dataset= data.values
training_data_len = math.ceil(len(dataset)*.8)
training_data_len

2561

In [26]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

array([[0.00000000e+00],
       [1.87796187e-06],
       [1.61743537e-06],
       ...,
       [2.73732729e-01],
       [2.75317101e-01],
       [2.74260525e-01]])

In [29]:
train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train=[]

for i in range(60, len(train_data)):
  x_train.append(train_data[i-60:i,0])
  y_train.append(train_data[i,0])
  if i<=61:
    print(x_train)
    print(y_train)
    print()

[array([0.00000000e+00, 1.87796187e-06, 1.61743537e-06, 1.30418326e-06,
       1.53524546e-06, 5.11748486e-08, 6.77679056e-07, 2.60009584e-07,
       5.11748486e-08, 3.35479563e-07, 5.42246628e-07, 4.85385685e-07,
       1.05399511e-06, 6.81814397e-07, 9.48026993e-07, 5.99107571e-07,
       5.42246628e-07, 5.42246628e-07, 3.87171329e-07, 5.93938394e-07,
       6.61137691e-07, 4.90554862e-07, 5.88769218e-07, 1.11085606e-06,
       1.05916429e-06, 9.04088992e-07, 1.05916429e-06, 7.74859576e-07,
       9.04088992e-07, 8.15696072e-07, 8.26551343e-07, 1.05916429e-06,
       9.55780758e-07, 8.88581462e-07, 8.26551343e-07, 8.73073932e-07,
       8.52397226e-07, 7.96053200e-07, 8.00705459e-07, 7.90367106e-07,
       7.49013693e-07, 8.00705459e-07, 7.54182870e-07, 7.49013693e-07,
       7.99154706e-07, 5.42246628e-07, 6.92152750e-07, 7.17998633e-07,
       5.86184629e-07, 6.65273032e-07, 6.24953454e-07, 6.24953454e-07,
       5.93938394e-07, 6.45630161e-07, 5.99624489e-07, 6.35291807e-07,
     

In [30]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [31]:
x_train =np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
x_train.shape

(2501, 60, 1)

In [32]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))


In [33]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [34]:
model.fit(x_train, y_train, batch_size=1, epochs=1)

2501/2501 [==============================] - 72s 27ms/step - loss: 4.8300e-05


In [36]:
test_data = scaled_data[training_data_len-60:,:]
x_test=[]
y_test=dataset[training_data_len:,:]
for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i, 0])

In [37]:
x_test = np.array(x_test)

In [38]:
x_test= np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

In [39]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)


In [40]:
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

2474.6987258251465

In [42]:
train= data[:training_data_len]
valid= data[training_data_len:]
valid['Predictions']= predictions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [43]:
valid

,Close,Predictions
2561,2675.080078,2835.484131
2562,2836.530029,2948.375732
2563,2756.610107,3074.248047
2564,2763.419922,3176.227783
2565,2582.580078,3256.086426
...,...,...
3196,5236.259766,6730.354492
3197,5289.750000,6754.932617
3198,5295.529785,6792.410156
3199,5326.180176,6832.020020


In [60]:
new_df= df.filter(['Close'])
last_60_days = new_df[-60:].values
last_60_days_scaled=scaler.transform(last_60_days)
X_test = []
X_test.append(last_60_days_scaled)
X_test = np.array(X_test)
X_test= np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
pred_price = model.predict(X_test)
pred_price=scaler.inverse_transform(pred_price)
print(pred_price)

[[6907.0327]]


In [61]:
# Create a Pickle file  
import pickle
pickle_out = open("model.pkl","wb")
pickle.dump(model, pickle_out)
pickle_out.close()

In [87]:
with open('model.pkl', 'rb') as f:
    x = pickle.load(f)

